In [1]:
import numpy as np
file_list = []
for i in range(1, 11):
    file_list.append('../tmp/hidden_{}.npy'.format(i))
xb = []
for file in file_list:
    x = np.load(file)
    xb.append(x)
xb = np.concatenate(xb, axis=0)
xb = xb.astype('float32')

In [2]:
import faiss
import time
import math
ngpu = 8
gpu_resources = []
tempmem = -1

for i in range(ngpu):
    res = faiss.StandardGpuResources()
    if tempmem >= 0:
        res.setTempMemory(tempmem)
    gpu_resources.append(res)

def make_vres_vdev(i0=0, i1=-1):
    " return vectors of device ids and resources useful for gpu_multiple"
    vres = faiss.GpuResourcesVector()
    vdev = faiss.IntVector()
    if i1 == -1:
        i1 = ngpu
    for i in range(i0, i1):
        vdev.push_back(i)
        vres.push_back(gpu_resources[i])
    return vres, vdev

nq = 1024
nb, d = xb.shape
print(nb, d)
co = faiss.GpuMultipleClonerOptions()
co.useFloat16 = True
co.useFloat16CoarseQuantizer = False
co.usePrecomputed = False
co.indicesOptions = 0
co.verbose = True
co.shard = True

res = faiss.StandardGpuResources()
start_time = time.time()
centroid_num = int(math.sqrt(nb) * 10)
print(centroid_num)
index = faiss.index_factory(d, "OPQ64_256,IVF{},PQ64".format(centroid_num))
vres, vdev = make_vres_vdev()
index = faiss.index_cpu_to_gpu_multiple(vres, vdev, index, co)
index.train(xb)
index.add(xb)




Loading faiss with AVX2 support.


124784384 410
111706


In [3]:
print('build index time:{}'.format(time.time() - start_time))
start_time = time.time()
k = 1024                          
D, I = index.search(xb[:nq], k)
print('query time:{}'.format(time.time() - start_time))

build index time:2209.1777305603027
query time:0.48778700828552246


In [4]:
cpu_index = faiss.index_gpu_to_cpu(index)

In [5]:
faiss.write_index(cpu_index, 'wt103.index')

In [13]:
test_index = faiss.read_index('test.index')
start_time = time.time()
k = 1024                          
D, I = test_index.search(xb[:nq], k)
print('query time:{}'.format(time.time() - start_time))

query time:0.04770302772521973
